# Visitor: classic visitor

> The canonical implementation of the visitor design pattern

The "canonical" implementation (as shown in the GoF book) of the Visitor Design Pattern makes use of a technique called **double dispatch**, which we'll soon see.

We will begin with the same basis as the previous lesson: We will have our expression classes and we'll have a separate class for printing. This time, the `ExpressionPrinter` class will contain its own buffer that we will create when initializing it.

In [1]:
class DoubleExpression:
    def __init__(self, value):
        self.value = value

class AdditionExpression:
    def __init__(self, left, right):
        self.right = right
        self.left = left
        
class ExpressionPrinter:
    def __init__(self):
        self.buffer = []
        
    def __str__(self):
        ''.join(self.buffer)

We now want `ExpressionPrinter` to be able to take any kind of epxression and get it to print.

What we will do now is create a series of methods that will contain the code for printing one expression or another, depending on the type of expression received as an argument. We'll call these methods `visit` because they're the **visitor methods** that know how to traverse each data structured (each kind of expression), and the convention is to call these methods `visit`, but essentially they will behave like our previous `print` implementation for each expression type.

In [2]:
class ExpressionPrinter:
    def __init__(self):
        self.buffer = []
        
    def __str__(self):
        return ''.join(self.buffer)
        
    def visit(self, de):    # de = DoubleExpression
        self.buffer.append(str(de.value))
        
    def visit(self, ae):    # ae = AdditionExpression
        self.buffer.append('(')
        ae.left.accept(self)
        self.buffer.append('+')
        ae.right.accept(self)
        self.buffer.append(')')

However, the code above is wrong: we're re-declarating `visit` a second time, essentially overwriting the first method. Also, there's this weird `accept` method in the second definition of the method which we have not defined yet.

Let's deal with the `accept` method first. This will be a very simple method that we need to add to our expressions; we could either add it to each expression, or better yet, we can create a base class whith the method and inherit it.

In [3]:
from abc import ABC

class Expression(ABC):
    def accept(self, visitor):
        visitor.visit(self)
        
class DoubleExpression(Expression):
    def __init__(self, value):
        self.value = value

class AdditionExpression(Expression):
    def __init__(self, left, right):
        self.right = right
        self.left = left

`accept` receives ("accepts") a **visitor** (`ExpressionPrinter`), and then calls the `visit` method of the visitor passing itself as an argument.

This seems convoluted, but this essentially allows us to separate the printing operation entirely from the data structures (the expression classes), and have each class call its printing operation without having to modify anything in our classes.

This is called **double dispatch** because there are 2 objects that determine the method that gets called. In our example, `Expression.accept(visitor)` calls a method but what method exactly depends on `Expression` (which could be either `DoubleExpression` or `AdditionExpression`), and `visitor` (`ExpressionPrinter`) contains the behavior. We can create additional expressions such as subtraction and we could create additional visitors like for evaluating expressions, and the combination of the two parts would decide the correct code.

However, we still need to take care of the `visit` overwriting issue within `ExpressionPrinter`. The easiest way to deal with this would be to rename each `visit` method to something different for each expression type and then override the `accept` method within each expression class to call the appropiate method, but what we will do instead is create a **decorator** that we can use to differentiate each `visit` method inside `ExpressionPrinter`.

In [4]:
# taken from https://tavianator.com/the-visitor-pattern-in-python/

def _qualname(obj):
    """Get the fully-qualified name of an object (including module)."""
    return obj.__module__ + '.' + obj.__qualname__

def _declaring_class(obj):
    """Get the name of the class that declared an object."""
    name = _qualname(obj)
    return name[:name.rfind('.')]

# Stores the actual visitor methods
_methods = {}

# Delegating visitor implementation
def _visitor_impl(self, arg):
    """Actual visitor method implementation."""
    method = _methods[(_qualname(type(self)), type(arg))]
    return method(self, arg)

# The actual @visitor decorator
def visitor(arg_type):
    """Decorator that creates a visitor method."""

    def decorator(fn):
        declaring_class = _declaring_class(fn)
        _methods[(declaring_class, arg_type)] = fn

        # Replace all decorated methods with _visitor_impl
        return _visitor_impl

    return decorator

In [5]:
class ExpressionPrinter:
    def __init__(self):
        self.buffer = []
        
    def __str__(self):
        return ''.join(self.buffer)
    
    @visitor(DoubleExpression)
    def visit(self, de):
        self.buffer.append(str(de.value))
    
    @visitor(AdditionExpression)
    def visit(self, ae):
        self.buffer.append('(')
        ae.left.accept(self)
        self.buffer.append('+')
        ae.right.accept(self)
        self.buffer.append(')')

And that's it. Let's test our code:

In [7]:
# represents 1+(2+3)
e = AdditionExpression(
    DoubleExpression(1),
    AdditionExpression(
        DoubleExpression(2),
        DoubleExpression(3)
    )
)

printer = ExpressionPrinter()
printer.visit(e)
print(printer)

(1+(2+3))


In [ ]:
# represents 1+(2+3)
e = AdditionExpression(
    DoubleExpression(1),
    AdditionExpression(
        DoubleExpression(2),
        DoubleExpression(3)
    )
)

printer = ExpressionPrinter()
e.accept(printer)
print(printer)

(1+(2+3))


Note that it's now possible to invoke printing from either the printing class (by visiting our expression) or from the expression (by accepting the printing class from the expression). The "classical" version is to use the second version, which is element-driven and is more open for extension.